In [ ]:
source("yaara7.r")

In [ ]:
#conf_yaml_path <- "/home/dcsoft/s/yaara/configure yaml/configure MM15-127 cnv NEW snps3.yml"
#conf_yaml_path <- "/home/dcsoft/s/yaara/configure yaml/configure MM16-412 cnv NEW.yml"
#conf_yaml_path <- "/home/dcsoft/s/yaara/configure yaml/configure MM16-423 cnv NEW.yml"
#conf_yaml_path <- "/home/dcsoft/s/yaara/configure yaml/configure BC14-2680 cnv NEW.yml"
#conf_yaml_path <- "/home/dcsoft/s/yaara/configure yaml/configure BC15-0267 cnv NEW.yml"
#conf_yaml_path <- "/home/dcsoft/s/yaara/configure yaml/configure BC16-0401 cnv NEW.yml"
#conf_yaml_path <- "/home/dcsoft/s/yaara/configure yaml/configure BC16-0545 cnv NEW.yml"
#conf_yaml_path <- "/home/dcsoft/s/yaara/configure yaml/configure 775-13 cnv NEW.yml"

#775-13 MM15-127 MM16-412 MM16-423 BC14-2680 BC15-0267 BC15-207 BC16-0401 BC16-0545
#conf_yaml_path <- "/home/dcsoft/s/yaara/configure yaml/configure MM16-423 cnv NEW snps3.yml"
#conf_yaml_path <- "/home/dcsoft/s/yaara/configure yaml/configure MM16-412 cnv NEW snps3.yml"

#conf_yaml_path <- "/home/dcsoft/s/yaara/configure yaml/configure MM15-127 cnv NEW snps4.yml"
#conf_yaml_path <- "/home/dcsoft/s/yaara/configure yaml/configure MM16-423 cnv NEW snps4.yml"
#conf_yaml_path <- "/home/dcsoft/s/yaara/configure yaml/configure MM16-412 cnv NEW snps4.yml"

#conf_yaml_path <- "/home/dcsoft/s/yaara/configure yaml/configure karl 1.yml"
#conf_yaml_path <- "/home/dcsoft/s/yaara/configure yaml/configure 795-09 cnv snps1.yml"
#conf_yaml_path <- "/home/dcsoft/s/yaara/configure yaml/configure MM15-127 cnv NEW snps5.yml"

#conf_yaml_path <- "/home/dcsoft/s/yaara/configure yaml/configure MM15-127 cnv NEW snps with new cells.yml"

#conf_yaml_path <- "/home/dcsoft/s/yaara/configure yaml/configure 440 new a4.yml"
#conf_yaml_path <- "/home/dcsoft/s/yaara/configure yaml/configure 465 new.yml"

#conf_yaml_path <- "/home/dcsoft/s/yaara/configure yaml/configure BC14-2680 cnv NEW a4.yml"
#conf_yaml_path <- "/home/dcsoft/s/yaara/configure yaml/configure BC15-0267 cnv NEW a4.yml"
#conf_yaml_path <- "/home/dcsoft/s/yaara/configure yaml/configure BC16-0545cnv NEW a4.yml"
#conf_yaml_path <- "/home/dcsoft/s/yaara/configure yaml/configure 795-09 cnv NEW a4.yml"

# conf_yaml_path <- "/home/ofirr/s/Ofir/yamls/configure 795-09 cnv NEW a4.yml"
conf_yaml_path <- "/home/ofirr/s/Ofir/yamls/configure BC15-0267 cnv NEW a4.yml"
# conf_yaml_path <- "/home/ofirr/s/Ofir/yamls/configure MM15-127 cnv NEW a4.yml"

In [ ]:
#frame parameters
#for 230
#width <- 80
#height <- 40

#for 465
width <- 170*3
height <- 60*3

#for 440
#width <- 100
#height <- 50

#for eye
#width <- 60
#height <-25

# for MM15-127
#width <- 430
#height <- 250

# for karolinska big
#width <- 9000
#height <- 200

#width <- 220
#height <- 120

In [ ]:
id_tip_trim_flag <- TRUE
id_tip_prefix <- "ID"
id_tip_trim_start <- 3
id_tip_trim_end <-7
man_multiply_second_legend <- 0.54 #2.5
man_multiply_second_legend_text <-3
man_multiply_first_legend_text <-1 #6
man_multiply_first_legend_title_size <- 2
man_space_second_legend = 10
man_space_second_legend_multiplier=0.1
man_offset_for_highlight_legend_x=-6
a_4_output<- TRUE
man_adjust_image_of_second_legend <- 1.8

In [ ]:
out <- func.print.lineage.tree(conf_yaml_path,simulate.p.value=FALSE,
                               height= height,width = width, 
                               man_adjust_image_of_second_legend=man_adjust_image_of_second_legend, man_adj_heat_loc = 0,
                               man_boot_x_offset= 0.025,man_adj_heat_loc2=3, man_adj_heat_loc3=6,
                               id_tip_trim_flag= id_tip_trim_flag, 
                               id_tip_trim_start= id_tip_trim_start,
                               id_tip_trim_end=id_tip_trim_end,
                               id_tip_prefix = id_tip_prefix,
                               debug_mode=FALSE,
                               debug_print_data_tree= FALSE,
                               man_multiply_second_legend= man_multiply_second_legend,
                               man_multiply_second_legend_text= man_multiply_second_legend_text,
                               man_adjust_elipse=0.25, #11.05
                               man_multiply_elipse=14.5,
                               man_multiply_first_legend_text= man_multiply_first_legend_text,
                               man_multiply_first_legend_title_size= man_multiply_first_legend_title_size,
                               man_space_second_legend=man_space_second_legend,
                               man_space_second_legend_multiplier=man_space_second_legend_multiplier,
                               man_offset_for_highlight_legend_x = man_offset_for_highlight_legend_x,
                               a_4_output= a_4_output)
